In [16]:
from pathlib import Path
from shapely.geometry import Polygon

import geopandas as gpd
import huracanpy
import numpy as np
import pandas as pd
from parse import parse
from tqdm import tqdm
import xarray as xr

In [2]:
gws = Path("/gws/pw/j07/workshop/users/ukcgfi-hackathon/")

depresys = gws / "data/decadal/depresys/TRACKS/"
depresys_glob = "DePreSys4_*_*_*"
depresys_extra = "DePreSys4_{run_id}_{start_year:04d}_{ensemble_member:d}"
fname = "tr_trs_pos.2day_addT63vor_addw10m_addmslp_addprecip.tcident.new"

depresys_full = str(depresys / depresys_extra / fname)

variable_names = [
    "vorticity850hPa",
    "vorticity700hPa",
    "vorticity600hPa",
    "vmax10m",
    "mslp",
    "precip"
]

In [3]:
all_files = sorted([str(f) for f in depresys.rglob(depresys_glob + "/" + fname)])

In [ ]:
# Merge all tracks to a single netCDF
for lead_years in range(2, 12):
    print(lead_years)
    all_tracks = []
    for filename in tqdm(all_files):
        try:
            tracks = huracanpy.load(filename, variable_names=variable_names, source="TRACK")
        except ValueError:
            tracks = huracanpy.load(filename, variable_names=variable_names, source="TRACK", track_calendar="360_day")
            dt = tracks.time - tracks.time[0]
            tracks["time"] = tracks.time[0].astype("datetime64[s]") + dt

        info = parse(depresys_full, filename).named

        tracks = tracks.isel(record=np.where(tracks.time.dt.year == info["start_year"] + lead_years)[0])
    
        all_tracks.append(tracks.assign(
            start_year=("record", [info["start_year"]] * len(tracks.time)),
            ensemble_member=("record", [info["ensemble_member"]] * len(tracks.time)),
        ))
    
    all_tracks = huracanpy.concat_tracks(all_tracks)
    huracanpy.save(all_tracks, f"tracks/depresys/depresys_tcs_{lead_years:02d}-year-lead.nc")

2


100%|██████████| 630/630 [11:35<00:00,  1.10s/it]


3


100%|██████████| 630/630 [11:28<00:00,  1.09s/it]


4


100%|██████████| 630/630 [11:37<00:00,  1.11s/it]


5


100%|██████████| 630/630 [12:17<00:00,  1.17s/it]


6


 61%|██████▏   | 387/630 [07:01<04:30,  1.11s/it]

In [103]:
japan = pd.read_csv("~/john/Areas_JKH/Japan_1.0.csv", delimiter=" ", header=None, names=["lon", "lat"])
usa = pd.read_csv("~/john/Areas_JKH/USA_all.csv", delimiter=" ", header=None, names=["lon", "lat"])

usa_gulf_mexico = pd.read_csv("~/john/Areas_JKH/USA1_gulf_mexico.csv", delimiter=" ", header=None, names=["lon", "lat"])
usa_florida = pd.read_csv("~/john/Areas_JKH/USA2_florida_1.1.csv", delimiter=" ", header=None, names=["lon", "lat"])
usa_north = pd.read_csv("~/john/Areas_JKH/USA3_North.csv", delimiter=" ", header=None, names=["lon", "lat"])

B = dict(
    Japan=Polygon([(lon, lat) for lon, lat in zip(japan.lon, japan.lat)]),
    USA=Polygon([(lon, lat) for lon, lat in zip(usa.lon, usa.lat)]),
)
huracanpy.basins["CGFI"] = gpd.GeoDataFrame(index=B.keys(), geometry=list(B.values()))

B = dict(
    usa_gulf_mexico=Polygon([(lon, lat) for lon, lat in zip(usa_gulf_mexico.lon, usa_gulf_mexico.lat)]),
    usa_florida=Polygon([(lon, lat) for lon, lat in zip(usa_florida.lon, usa_florida.lat)]),
    usa_north=Polygon([(lon, lat) for lon, lat in zip(usa_north.lon, usa_north.lat)]),
)
huracanpy.basins["cgfi_gate"] = gpd.GeoDataFrame(index=B.keys(), geometry=list(B.values()))

from huracanpy.info import _geography
from importlib import reload

reload(_geography)

<module 'huracanpy.info._geography' from '/home/users/train187/miniforge3/envs/core/lib/python3.13/site-packages/huracanpy/info/_geography.py'>

In [104]:
data_out = []

convention = "cgfi_gate"
basins = ["usa_gulf_mexico", "usa_florida", "usa_north"]

# Loop over all forecasts
for n in range(1, 10+1):
    tracks = huracanpy.load(f"tracks/depresys/depresys_tcs_{n:02d}-year-lead.nc")
    tracks = tracks.hrcn.add_basin(convention=convention)
    tracks.wind[tracks.wind == 1e25] = 0.

    # Only use time at max intensity for counts
    tracks_apex = tracks.hrcn.get_apex_vals("vmax10m")

    # Only TC season
    tracks = tracks.hrcn.sel_id(
        tracks.track_id[(tracks.time.dt.month >= 6) & (tracks.time.dt.month <= 11)]
    )

    # Get cyclone counts by basin
    for basin in basins:
        ids_basin = tracks_apex.track_id[tracks_apex.basin == basin]
        apex_basin = tracks.hrcn.sel_id(ids_basin)
        tcs_basin = tracks.hrcn.sel_id(ids_basin)

        mask = (tcs_basin.basin == basin).astype(int)
        
        wind = tcs_basin.vmax10m * mask
        mslp = tcs_basin.mslp * mask
        
        ace = huracanpy.tc.ace(wind, threshold=0, sum_by=tracks.track_id)
        pace, _ = huracanpy.tc.pace(mslp, model="z2021")
        ssi = (wind ** 3).groupby("track_id").sum()
        counts = apex_basin.basin.groupby(tcs_basin.time.dt.year).count()

        data_out.append(
            pd.DataFrame(data=dict(
                start_year=[info["start_year"]] * len(counts.year),
                ensemble_member=[info["ensemble_member"]] * len(counts.year),
                year=counts.year.values,
                basin=[basin] * len(counts.year),
                count=counts.values,
                ace=ace.groupby(tcs_basin.time.dt.year).sum().values,
                pace=pace.groupby(tcs_basin.time.dt.year).sum().values,
                ssi=ssi.groupby(tcs_basin.time.dt.year).sum().values,
            ))
        )

        data_out.to_csv(f"depresys_tc_activity_metrics_lead_time_{n}_basin_{basin}.csv", index=False)

AttributeError: 'Dataset' object has no attribute 'wind'

In [105]:
convention = "cgfi_gate"
basins = ["usa_gulf_mexico", "usa_florida", "usa_north"]

# Loop over all forecasts
for n in tqdm(range(1, 10+1)):
    tracks = huracanpy.load(f"tracks/depresys/depresys_tcs_{n:02d}-year-lead.nc")
    tracks = tracks.hrcn.add_basin(convention=convention)
    tracks.vmax10m[tracks.vmax10m == 1e25] = 0.

    # Only use time at max intensity for counts
    tracks_apex = tracks.hrcn.get_apex_vals("vmax10m")
    
    # Only TC season
    tracks = tracks.hrcn.sel_id(
        tracks.track_id[(tracks.time.dt.month >= 6) & (tracks.time.dt.month <= 11)]
    )

    # Get cyclone counts by basin
    for basin in basins:
        ids_basin = tracks_apex.track_id[tracks_apex.basin == basin]
        apex_basin = tracks_apex.hrcn.sel_id(ids_basin)
        tcs_basin = tracks.hrcn.sel_id(ids_basin)
        
        mask = (tcs_basin.basin == basin).astype(int)
        
        wind = tcs_basin.vmax10m * mask
        mslp = tcs_basin.mslp * mask
        
        ace = huracanpy.tc.ace(wind, threshold=0, sum_by=tcs_basin.track_id)
        pace, _ = huracanpy.tc.pace(mslp, model="z2021", sum_by=tcs_basin.track_id)
        ssi = (wind ** 3).groupby(tcs_basin.track_id).sum()
        precip = (tcs_basin.precip * mask).groupby(tcs_basin.track_id).sum()

        results = xr.merge([dict(ace=ace, pace=pace, ssi=ssi, precip=precip)])
        results = xr.merge([results, apex_basin[["start_year", "ensemble_member", "time"]]])
        results = results.sortby(["start_year", "ensemble_member", "time"])

        results_sum = results.groupby(results.start_year.astype(str) + results.ensemble_member.astype(str)).sum()
        results_mean = results.groupby(results.start_year.astype(str) + results.ensemble_member.astype(str)).mean()
        results_count = results.groupby(results.start_year.astype(str) + results.ensemble_member.astype(str)).count()

        results_all = xr.merge([
            results_sum[["ace", "pace", "ssi", "precip"]],
            results_mean["start_year"].astype(int),
            results_mean["ensemble_member"].astype(int),
            results_count.ace.rename("count"),
        ])

        results_csv = results_all.drop_vars("group").to_pandas()

        results_csv_all = []
        for start_year in range(1960, 2022+1):
            for ensemble_member in range(1, 10):
                row = results_csv[
                    (results_csv.start_year == start_year) &
                    (results_csv.ensemble_member == ensemble_member)
                ]
        
                if len(row) == 1:
                    results_csv_all.append(row)
                else:
                    results_csv_all.append(pd.DataFrame(
                        dict(ace=[0], pace=[0], ssi=[0], precip=[0], start_year=[start_year], ensemble_member=[ensemble_member], count=[0])
                    ))

        pd.concat(results_csv_all).to_csv(f"depresys_tc_activity_metrics_lead_time_{n}_basin_{basin}.csv", index=False)

100%|██████████| 10/10 [03:39<00:00, 21.95s/it]
